In [18]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from torch.utils.data import Dataset, DataLoader
import os
from typing import Tuple, List, Type, Dict, Any


In [19]:
with open('./index.pkl', 'rb') as f:
    data_index = pickle.load(f)

In [20]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print('Using GPU', f'({torch.cuda.get_device_name()})')
else:
    device = torch.device('cpu')
    print('Using CPU')

Using GPU (GeForce MX150)


In [21]:
data_index[1]

{'jpg_filename': 'img-2019-12-11T09-56-17devID2.jpg',
 'lon': -10.775719833333333,
 'lat': 41.35916666666667,
 'mask_fname': 'AMK79/masks/mask-id2.png',
 'mission': 'AMK79',
 'devID': 2,
 'observations_dt': datetime.datetime(2019, 12, 11, 10, 0),
 'observed_TCC': 4}

In [22]:
data = pd.DataFrame(data_index)

In [23]:
data

,jpg_filename,lon,lat,mask_fname,mission,devID,observations_dt,observed_TCC
0,img-2019-12-11T10-02-37devID2.jpg,-10.783049,41.343315,AMK79/masks/mask-id2.png,AMK79,2,2019-12-11 10:00:00.000,4
1,img-2019-12-11T09-56-17devID2.jpg,-10.775720,41.359167,AMK79/masks/mask-id2.png,AMK79,2,2019-12-11 10:00:00.000,4
2,img-2019-12-11T10-04-17devID1.jpg,-10.784958,41.339135,AMK79/masks/mask-id1.png,AMK79,1,2019-12-11 10:00:00.000,4
3,img-2019-12-11T14-00-00devID2.jpg,-11.037792,40.761531,AMK79/masks/mask-id2.png,AMK79,2,2019-12-11 13:59:59.975,1
4,img-2019-12-11T12-58-20devID2.jpg,-10.973738,40.909492,AMK79/masks/mask-id2.png,AMK79,2,2019-12-11 12:59:59.980,1
...,...,...,...,...,...,...,...,...
92072,img-2016-01-03T13-56-08devID1.jpg,23.872334,33.830525,ANS31/masks/mask-id1.png,ANS31,1,2016-01-03 14:00:00.000,7
92073,img-2016-01-03T07-01-23devID1.jpg,25.357262,33.476699,ANS31/masks/mask-id1.png,ANS31,1,2016-01-03 07:00:02.000,5
92074,img-2016-01-03T14-58-53devID1.jpg,23.635267,33.886403,ANS31/masks/mask-id1.png,ANS31,1,2016-01-03 15:00:04.000,1
92075,img-2016-01-03T07-00-53devID2.jpg,25.359043,33.476318,ANS31/masks/mask-id2.png,ANS31,2,2016-01-03 07:00:02.000,5


In [24]:
dataframe = data[['jpg_filename', 'mission', 'observations_dt', 'observed_TCC']]

In [25]:
dataframe

,jpg_filename,mission,observations_dt,observed_TCC
0,img-2019-12-11T10-02-37devID2.jpg,AMK79,2019-12-11 10:00:00.000,4
1,img-2019-12-11T09-56-17devID2.jpg,AMK79,2019-12-11 10:00:00.000,4
2,img-2019-12-11T10-04-17devID1.jpg,AMK79,2019-12-11 10:00:00.000,4
3,img-2019-12-11T14-00-00devID2.jpg,AMK79,2019-12-11 13:59:59.975,1
4,img-2019-12-11T12-58-20devID2.jpg,AMK79,2019-12-11 12:59:59.980,1
...,...,...,...,...
92072,img-2016-01-03T13-56-08devID1.jpg,ANS31,2016-01-03 14:00:00.000,7
92073,img-2016-01-03T07-01-23devID1.jpg,ANS31,2016-01-03 07:00:02.000,5
92074,img-2016-01-03T14-58-53devID1.jpg,ANS31,2016-01-03 15:00:04.000,1
92075,img-2016-01-03T07-00-53devID2.jpg,ANS31,2016-01-03 07:00:02.000,5


In [26]:
class SkyData(Dataset):
    
    def __init__(self, data_list, root_dir, augment = None):
        
        super().__init__()
        
        self.data_list = data_list
        self.root_dir = os.path.abspath(root_dir)
        self.augment = augment
        
    def __len__(self):
        
        return len(self.annotations)
        
    def __getitem__(self, index):
        
        img_path = os.path.join(self.root_dir, self.data_list.iloc[index]['mission'], 'snapshots', 'snapshots-'+str((self.data_list.iloc[index]['observations_dt']).date()), self.data_list.iloc[index]['jpg_filename'])
        image = plt.imread(img_path)
        label = torch.tensor(int(self.data_list.iloc[index]['observed_TCC']))
        
        if self.augment:
            image = self.augment(image)
    
            
        return (image, label)

In [27]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [28]:
SkyData = SkyData(dataframe, root_dir = 'geo_kaggle', augment = transforms)

In [29]:
SkyData[254]

(tensor([[[0.0627, 0.0510, 0.0353,  ..., 0.1412, 0.1255, 0.1569],
          [0.0627, 0.0627, 0.0510,  ..., 0.1373, 0.1176, 0.1529],
          [0.0588, 0.0706, 0.0627,  ..., 0.1451, 0.1373, 0.1647],
          ...,
          [0.0667, 0.0745, 0.0745,  ..., 0.0039, 0.0078, 0.0157],
          [0.0667, 0.0745, 0.0667,  ..., 0.0118, 0.0118, 0.0078],
          [0.0667, 0.0745, 0.0667,  ..., 0.0196, 0.0078, 0.0078]],
 
         [[0.0745, 0.0588, 0.0431,  ..., 0.0353, 0.0196, 0.0588],
          [0.0706, 0.0706, 0.0588,  ..., 0.0314, 0.0196, 0.0549],
          [0.0667, 0.0784, 0.0706,  ..., 0.0471, 0.0392, 0.0706],
          ...,
          [0.0863, 0.0941, 0.0824,  ..., 0.0549, 0.0667, 0.0706],
          [0.0863, 0.0941, 0.0863,  ..., 0.0627, 0.0706, 0.0745],
          [0.0863, 0.0941, 0.0863,  ..., 0.0745, 0.0745, 0.0784]],
 
         [[0.0471, 0.0392, 0.0314,  ..., 0.0549, 0.0392, 0.0706],
          [0.0510, 0.0510, 0.0471,  ..., 0.0510, 0.0314, 0.0667],
          [0.0471, 0.0588, 0.0588,  ...,

In [30]:
def train_single_epoch(model: torch.nn.Module,
                       optimizer: torch.optim.Optimizer, 
                       loss_function: torch.nn.Module, 
                       data_loader: torch.utils.data.DataLoader):
    
    model.train()
    loss_sum = 0

    for data in data_loader:
        x, y = data
        x, y = x.to(device), y.to(device)

        model.zero_grad()
        hyp = model(x)
        loss = loss_function(hyp, y)
        loss.backward()
        loss_sum += loss
        
        optimizer.step()

    
    return loss_sum / len(data_loader.dataset)


In [31]:
def validate_single_epoch(model: torch.nn.Module,
                          loss_function: torch.nn.Module, 
                          data_loader: torch.utils.data.DataLoader):
    model.eval()
    loss_sum = 0
    accuracy = 0

    for data in data_loader:
        x, y = data
        x, y = x.to(device), y.to(device)

        hyp = model(x)
        loss = loss_function(hyp, y)
        loss_sum += loss

        y_pred = hyp.argmax(dim = 1, keepdim = True).to(device)
        accuracy += y_pred.eq(y.view_as(y_pred)).sum().item()

    loss_avr = loss_sum / len(data_loader.dataset)
    accuracy_avr = 100 * accuracy / len(data_loader.dataset)
    
    return {'loss' : loss_avr.item(), 'accuracy' : accuracy_avr}

In [32]:
def ploting_curves(loss, best_epoch):
    """
    Plot loss evolution on training and validation sets
    """
    # Plot learning loss curve
    plt.plot(loss['train'], label = 'Training set')
    plt.plot(loss['valid'], label = 'Val set')
    plt.axvline(best_epoch, color = 'r', ls = '--', label = 'Best model')
    plt.title('Loss evolution')
    plt.xlabel('epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [33]:
def train_model(model: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                val_dataset: torch.utils.data.Dataset,
                loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                initial_lr = 0.01,
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ReduceLROnPlateau,
                lr_scheduler_params: Dict = {},
                batch_size = 1024,
                max_epochs = 1000,
                early_stopping_patience = 20):
  
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    best_val_loss = None
    best_epoch = None
    loss_list = {'train': list(), 'valid': list()}
    

    for epoch in range(max_epochs):
        
        print(f'Epoch {epoch}')
        train_loss =  train_single_epoch(model, optimizer, loss_function, train_loader)
        loss_list['train'].append(train_loss)
        val_metrics = validate_single_epoch(model, loss_function, val_loader)
        loss_list['valid'].append(val_metrics['loss'])
        print(f'Validation metrics: \n{val_metrics}')

        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
            torch.save(model, './best_model.pth')
            
        if epoch - best_epoch > early_stopping_patience:
            print('Early stopping triggered')
            ploting_curves(loss_list,best_epoch)
            return

In [34]:
class Perceptron(torch.nn.Module):
    
    def __init__(self, 
                 input_resolution: Tuple[int, int] = (28, 28),
                 input_channels: int = 1, 
                 hidden_layer_features: List[int] = [256, 256, 256],
                 activation: Type[torch.nn.Module] = torch.nn.ReLU,
                 num_classes: int = 10):

        super().__init__()


        
        
    def forward(self, x):
      
        selu = self.activation()
        x = x.to(device)

        x = self.layer1(x)
        x = selu(torch.tensor(x))
        x = self.layer2(x)

        log_softmax = torch.nn.functional.log_softmax(x, dim = 1)

        return log_softmax

In [ ]:
model = Perceptron()
print(model)
print('Total number of trainable parameters', 
      sum(p.numel() for p in model.parameters() if p.requires_grad))